In [2]:
import polars as pl
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures,
)
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.tensorflow
import mlflow.keras
from mlflow.models.signature import infer_signature
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError as RMSE, R2Score

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

Num GPUs Available:  0


In [3]:
def get_data(poly_features: int = 1):
    # Get Data
    data = pl.read_parquet("data.parquet")
    data = data.drop(["Step", "Light_ID", "Lane", "Intersection_u", "Sim_ID"])
    data = data.with_columns(pl.col("Is_Entrypoint").cast(pl.Int8))
    print(f"Data: {data.shape}")
    print(f"{data.collect_schema()}")

    # Split Data
    X = data.drop("Num_Cars").to_numpy()
    y = data.select(pl.col("Num_Cars")).to_numpy()
    y = y.ravel()
    print("")
    print(f"X: {X.shape}")
    print(f"y: {y.shape}")

    # Scale
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Polynomial Features
    if poly_features > 1:
        poly = PolynomialFeatures(degree=poly_features)
        X = poly.fit_transform(X)

    # Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.6, test_size=0.4, random_state=42
    )
    print("")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test {y_test.shape}")

    # Train Test Validation Split
    X_test, X_val, y_test, y_val = train_test_split(
        X_test, y_test, train_size=0.5, test_size=0.5, random_state=42
    )
    print("")
    print(f"X_test: {X_test.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"y_test: {y_test.shape}")
    print(f"y_val: {y_val.shape}")

    return X_train, X_test, X_val, y_train, y_test, y_val

In [4]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

Data: (6610000, 5)
Schema({'Time': Int16, 'Num_Cars': Int16, 'Centrality': Float32, 'Is_Entrypoint': Int8, 'Distance': Int16})

X: (6610000, 4)
y: (6610000,)

X_train: (3966000, 4)
X_test: (2644000, 4)
y_train: (3966000,)
y_test (2644000,)

X_test: (1322000, 4)
X_val: (1322000, 4)
y_test: (1322000,)
y_val: (1322000,)


## Base Model

In [4]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")

Epoch 1/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 61s 491us/step - loss: 2.0758 - r2_score: 0.1582 - root_mean_squared_error: 1.4407 - val_loss: 2.0347 - val_r2_score: 0.1704 - val_root_mean_squared_error: 1.4264
Epoch 2/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 58s 468us/step - loss: 2.0360 - r2_score: 0.1726 - root_mean_squared_error: 1.4269 - val_loss: 2.0235 - val_r2_score: 0.1750 - val_root_mean_squared_error: 1.4225
Epoch 3/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 57s 462us/step - loss: 2.0249 - r2_score: 0.1759 - root_mean_squared_error: 1.4230 - val_loss: 2.0225 - val_r2_score: 0.1754 - val_root_mean_squared_error: 1.4221
Epoch 4/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 57s 463us/step - loss: 2.0172 - r2_score: 0.1783 - root_mean_squared_error: 1.4203 - val_loss: 2.0217 - val_r2_score: 0.1757 - val_root_mean_squared_error: 1.4219
Epoch 5/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 57s 461us/step - loss: 2.0200 - r2_score: 0.1795 - root_mean_squared_error: 1.4213 - val_loss: 2.0150 - val_r2_score: 

2025/04/11 18:09:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/11 18:09:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.013606548309326
RMSE: 1.4190160036087036
R2Score: 0.18041473627090454


## Add Layers

In [5]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")

Epoch 1/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 510us/step - loss: 2.0709 - r2_score: 0.1579 - root_mean_squared_error: 1.4390 - val_loss: 2.0284 - val_r2_score: 0.1730 - val_root_mean_squared_error: 1.4242
Epoch 2/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 516us/step - loss: 2.0369 - r2_score: 0.1721 - root_mean_squared_error: 1.4272 - val_loss: 2.0172 - val_r2_score: 0.1775 - val_root_mean_squared_error: 1.4203
Epoch 3/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 514us/step - loss: 2.0368 - r2_score: 0.1754 - root_mean_squared_error: 1.4271 - val_loss: 2.0235 - val_r2_score: 0.1750 - val_root_mean_squared_error: 1.4225
Epoch 4/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 515us/step - loss: 2.0173 - r2_score: 0.1770 - root_mean_squared_error: 1.4203 - val_loss: 2.0193 - val_r2_score: 0.1767 - val_root_mean_squared_error: 1.4210
Epoch 5/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 514us/step - loss: 2.0187 - r2_score: 0.1778 - root_mean_squared_error: 1.4208 - val_loss: 2.0167 - val_r2_score: 

2025/04/12 00:42:52 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/12 00:42:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.0207467079162598
RMSE: 1.4215296506881714
R2Score: 0.17750853300094604


In [6]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(256, activation="relu"),
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")


Epoch 1/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 2024s 16ms/step - loss: 2.0742 - r2_score: 0.1570 - root_mean_squared_error: 1.4402 - val_loss: 2.0292 - val_r2_score: 0.1726 - val_root_mean_squared_error: 1.4245
Epoch 2/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 1928s 16ms/step - loss: 2.0381 - r2_score: 0.1716 - root_mean_squared_error: 1.4276 - val_loss: 2.0222 - val_r2_score: 0.1755 - val_root_mean_squared_error: 1.4220
Epoch 3/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 1000s 8ms/step - loss: 2.0271 - r2_score: 0.1757 - root_mean_squared_error: 1.4238 - val_loss: 2.0219 - val_r2_score: 0.1756 - val_root_mean_squared_error: 1.4219
Epoch 4/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 1964s 16ms/step - loss: 2.0288 - r2_score: 0.1774 - root_mean_squared_error: 1.4244 - val_loss: 2.0145 - val_r2_score: 0.1786 - val_root_mean_squared_error: 1.4193
Epoch 5/10
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 384s 3ms/step - loss: 2.0251 - r2_score: 0.1775 - root_mean_squared_error: 1.4230 - val_loss: 2.0139 - val_r2_score

2025/04/12 04:49:10 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/12 04:49:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.018677234649658
RMSE: 1.4208016395568848
R2Score: 0.17835086584091187


In [7]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 15
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(256, activation="relu"),
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")

Epoch 1/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 82s 661us/step - loss: 2.0761 - r2_score: 0.1569 - root_mean_squared_error: 1.4408 - val_loss: 2.0386 - val_r2_score: 0.1688 - val_root_mean_squared_error: 1.4278
Epoch 2/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 82s 664us/step - loss: 2.0339 - r2_score: 0.1703 - root_mean_squared_error: 1.4261 - val_loss: 2.0366 - val_r2_score: 0.1696 - val_root_mean_squared_error: 1.4271
Epoch 3/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 82s 665us/step - loss: 2.0426 - r2_score: 0.1728 - root_mean_squared_error: 1.4292 - val_loss: 2.0247 - val_r2_score: 0.1745 - val_root_mean_squared_error: 1.4229
Epoch 4/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 83s 670us/step - loss: 2.0259 - r2_score: 0.1761 - root_mean_squared_error: 1.4233 - val_loss: 2.0216 - val_r2_score: 0.1758 - val_root_mean_squared_error: 1.4218
Epoch 5/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 84s 681us/step - loss: 2.0215 - r2_score: 0.1767 - root_mean_squared_error: 1.4218 - val_loss: 2.0154 - val_r2_score: 

2025/04/12 08:32:57 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/12 08:33:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.0284347534179688
RMSE: 1.4242312908172607
R2Score: 0.1743793487548828


In [8]:
OPTIMIZER = "Adam"
LR = 0.001
EPOCHS = 15
BATCH_SIZE = 32

with mlflow.start_run():
    model = Sequential(
        [
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")


Epoch 1/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 513us/step - loss: 2.0723 - r2_score: 0.1572 - root_mean_squared_error: 1.4395 - val_loss: 2.0382 - val_r2_score: 0.1690 - val_root_mean_squared_error: 1.4277
Epoch 2/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 508us/step - loss: 2.0323 - r2_score: 0.1734 - root_mean_squared_error: 1.4256 - val_loss: 2.0188 - val_r2_score: 0.1769 - val_root_mean_squared_error: 1.4209
Epoch 3/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 511us/step - loss: 2.0254 - r2_score: 0.1771 - root_mean_squared_error: 1.4232 - val_loss: 2.0189 - val_r2_score: 0.1768 - val_root_mean_squared_error: 1.4209
Epoch 4/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 511us/step - loss: 2.0211 - r2_score: 0.1778 - root_mean_squared_error: 1.4216 - val_loss: 2.0120 - val_r2_score: 0.1797 - val_root_mean_squared_error: 1.4185
Epoch 5/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 514us/step - loss: 2.0182 - r2_score: 0.1781 - root_mean_squared_error: 1.4206 - val_loss: 2.0150 - val_r2_score: 

2025/04/12 12:16:17 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/12 12:16:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.0164895057678223
RMSE: 1.4200315475463867
R2Score: 0.17924129962921143


In [9]:
OPTIMIZER = "Adam"
LR = 0.001
DECAY = 0.96
DECAY_STEPS = 100000
EPOCHS = 15
BATCH_SIZE = 32

with mlflow.start_run():
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=LR,
        decay_steps=DECAY_STEPS,
        decay_rate=DECAY,
        staircase=True,
    )

    model = Sequential(
        [
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")


Epoch 1/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 64s 511us/step - loss: 2.0749 - r2_score: 0.1574 - root_mean_squared_error: 1.4404 - val_loss: 2.0538 - val_r2_score: 0.1626 - val_root_mean_squared_error: 1.4331
Epoch 2/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 508us/step - loss: 2.0370 - r2_score: 0.1723 - root_mean_squared_error: 1.4272 - val_loss: 2.0286 - val_r2_score: 0.1729 - val_root_mean_squared_error: 1.4243
Epoch 3/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 508us/step - loss: 2.0279 - r2_score: 0.1750 - root_mean_squared_error: 1.4240 - val_loss: 2.0170 - val_r2_score: 0.1776 - val_root_mean_squared_error: 1.4202
Epoch 4/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 511us/step - loss: 2.0225 - r2_score: 0.1768 - root_mean_squared_error: 1.4221 - val_loss: 2.0138 - val_r2_score: 0.1789 - val_root_mean_squared_error: 1.4191
Epoch 5/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 63s 510us/step - loss: 2.0233 - r2_score: 0.1778 - root_mean_squared_error: 1.4224 - val_loss: 2.0265 - val_r2_score: 

2025/04/12 15:30:39 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/12 15:30:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.015489339828491
RMSE: 1.4196792840957642
R2Score: 0.17964839935302734


In [5]:
OPTIMIZER = "Adam"
LR = 0.001
DECAY = 0.96
DECAY_STEPS = 100000
EPOCHS = 15
BATCH_SIZE = 32

with mlflow.start_run():
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=LR,
        decay_steps=DECAY_STEPS,
        decay_rate=DECAY,
        staircase=True,
    )

    model = Sequential(
        [
            Dense(256, activation="relu"),
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.build(input_shape=(None, X_train.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=LR), loss="mse", metrics=[RMSE(), R2Score()]
    )

    mlflow.log_param("optimizer", OPTIMIZER)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
    )

    for epoch in range(EPOCHS):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric(
            "train_rmse", history.history["root_mean_squared_error"][epoch], step=epoch
        )
        mlflow.log_metric("train_r2", history.history["r2_score"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric(
            "val_rmse",
            history.history["val_root_mean_squared_error"][epoch],
            step=epoch,
        )
        mlflow.log_metric("val_r2", history.history["val_r2_score"][epoch], step=epoch)

    loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=0)

    mlflow.log_metric("test_loss", loss)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2_score)

    mlflow.keras.log_model(model, "model")

    print("")
    print(f"Loss: {loss}")
    print(f"RMSE: {rmse}")
    print(f"R2Score: {r2_score}")

Epoch 1/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 101s 811us/step - loss: 2.0753 - r2_score: 0.1567 - root_mean_squared_error: 1.4406 - val_loss: 2.0398 - val_r2_score: 0.1683 - val_root_mean_squared_error: 1.4282
Epoch 2/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 83s 666us/step - loss: 2.0345 - r2_score: 0.1719 - root_mean_squared_error: 1.4263 - val_loss: 2.0323 - val_r2_score: 0.1714 - val_root_mean_squared_error: 1.4256
Epoch 3/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 85s 683us/step - loss: 2.0333 - r2_score: 0.1753 - root_mean_squared_error: 1.4259 - val_loss: 2.0280 - val_r2_score: 0.1731 - val_root_mean_squared_error: 1.4241
Epoch 4/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 92s 741us/step - loss: 2.0278 - r2_score: 0.1764 - root_mean_squared_error: 1.4240 - val_loss: 2.0197 - val_r2_score: 0.1765 - val_root_mean_squared_error: 1.4212
Epoch 5/15
123938/123938 ━━━━━━━━━━━━━━━━━━━━ 92s 742us/step - loss: 2.0349 - r2_score: 0.1760 - root_mean_squared_error: 1.4265 - val_loss: 2.0242 - val_r2_score:

2025/04/13 13:48:23 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/13 13:48:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Loss: 2.0195181369781494
RMSE: 1.4210975170135498
R2Score: 0.17800861597061157
